In [1]:
import plots
import tables
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import spline
import json


SString = "StatResults/Sampled/"
sString = "StatResults/"
incon = "/InconsistencyStatistics.json"
kbStaaa = "/kbStatistics.json"

Colors = ['#3cb44b','#e6194b', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6','#ffe119', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']


In [2]:
FileListInconsistency = 	   [sString+'DBpedia'+incon,
								sString+'Yago'+incon,
								sString+'dblp-2012-11-28b'+incon]
FileListInconsistencySample =  [SString+'dbpedia2016-04en'+incon,
								SString+'yago2s'+incon,
								SString+'dblp-2012-11-28b'+incon]
FileListKB =            	   [sString+'DBpedia'+kbStaaa,
								sString+'Yago'+kbStaaa,
								sString+'dblp-2012-11-28b'+kbStaaa]
FileListKBSample =  			[SString+'dbpedia2016-04en'+kbStaaa,
								SString+'yago2s'+kbStaaa,
								SString+'dblp-2012-11-28b'+kbStaaa]


In [3]:
FileListInconsistency = 	   [sString+'DBpedia'+incon,
								sString+'Yago'+incon,
								sString+'wordnet'+incon,
								sString+'dblp-2012-11-28b'+incon,
								sString+'swdf'+incon,
								sString+'LOD'+incon]
FileListInconsistencySample =  [SString+'dbpedia2016-04en'+incon,
								SString+'yago2s'+incon,
								SString+'wordnet31'+incon,
								SString+'dblp-2012-11-28b'+incon,
								SString+'swdf'+incon]
FileListKB =            	   [sString+'DBpedia'+kbStaaa,
								sString+'Yago'+kbStaaa,
								sString+'wordnet'+kbStaaa,
								sString+'dblp-2012-11-28b'+kbStaaa,
								sString+'swdf'+kbStaaa]
FileListKBSample =  			[SString+'dbpedia2016-04en'+kbStaaa,
								SString+'yago2s'+kbStaaa,
								SString+'wordnet31'+kbStaaa,
								SString+'dblp-2012-11-28b'+kbStaaa,
								SString+'swdf'+kbStaaa]

In [4]:

def plotting(FileListInconsistency, FileListKB, Sample):
    counter = 0
    ColorScheme = {}
    labels1 = []
    labels2 = []
    width = (1-0.1)/len(FileListInconsistency)
    for file in FileListInconsistency:
        print(file)
        datasetName = file.split("/")[-2]
        ColorScheme[datasetName] = Colors[len(ColorScheme.keys())]
        print(ColorScheme[datasetName])
        labels1.append(datasetName)
        json_file = open(file, 'r')
        data = json.load(json_file)
        IDCoor = []
        Size = [i for i in range(0,25)]
        SizeCount = [0 for i in range(0,25)]
        SizeCoor = []
        SizeSum = [0 for i in range(0,25)]
        CountCoor = []
        for elem in data:
            CountCoor.append(elem["Count"])
            SizeCoor.append(elem["Size"])
            IDCoor.append(elem["id"])
            if(elem["Count"] > 0):
                SizeCount[elem["Size"]] += 1

            SizeSum[elem["Size"]] += elem["Count"]
        plt.figure(1)
        plt.title('Plot showing the Distribution of Size')
        plt.plot(Size, SizeCount, color=ColorScheme[datasetName])
        plt.ylabel('Count of different "Anti-pattern" types with the same size')
        plt.xlabel('Size of the "Anti-pattern"')

        plt.figure(2)

        SizeSum[:] = [x / float(sum(SizeSum)+0.0000001) for x in SizeSum]
        plt.title('Plot showing the normalised Distribution of Occurences')
        Size = np.arange(0,25)
        try:
            plt.bar(Size+width*counter, SizeSum, width, color=ColorScheme[datasetName])
        except:
            pass
        counter += 1
        plt.ylabel('Occurrences of the "Anti-pattern"')
        plt.xlabel('Size of the "Anti-pattern"')

        CountCoor.sort(reverse = True)
        CountCoor[:] = [x / float(sum(CountCoor)+0.0000001) for x in CountCoor]
        plt.figure(3)
        plt.title('Plot showing the normalised Distribution of Occurences')
        plt.plot( CountCoor, color=ColorScheme[datasetName])
        plt.ylabel('Occurrences of the "Anti-pattern"')
        plt.xlabel('"Anti-pattern" type')

    ColorScheme = {'dbpedia2016-04en': '#4363d8', 'yago2s': '#f58231', 'dblp-2012-11-28b': '#e6194b'} 
    for file in FileListKB:
        json_file = open(file, 'r')
        data = json.load(json_file)

        datasetName = file.split("/")[-2]
        labels2.append(datasetName)
        if not(datasetName in ColorScheme):
            ColorScheme[datasetName] = Colors[len(ColorScheme.keys())]
        XcoorIn = []
        YcoorIn = []
        for key in sorted(data["InDegree"].keys(), key=int):
            XcoorIn.append(int(key))
            YcoorIn.append(int(data["InDegree"][key]))

        plt.figure(4)
        plt.title('Plot showing the Distribution of the nodes with inlinks')
        plt.plot(XcoorIn, YcoorIn, color=ColorScheme[datasetName])
        plt.ylabel('Occurences of the nodes with inlinks')
        plt.xlabel('Amount of inlinks towards the node')


        XcoorOut = []
        YcoorOut = []
        for key in sorted(data["OutDegree"].keys(), key=int):
            XcoorOut.append(int(key))
            YcoorOut.append(int(data["OutDegree"][key]))

        plt.figure(5)
        plt.title('Plot showing the Distribution of the nodes with outlinks')
        plt.plot(XcoorOut, YcoorOut, color=ColorScheme[datasetName])
        plt.ylabel('Occurences of the nodes with equal outlinks')
        plt.xlabel('Amount of outlinks outwards the node')

        XcoorCluster = []
        YcoorCluster = []

        for key in sorted(data["ClusteringCoefficient"].keys(), key=float):
            XcoorCluster.append(float(key))
            YcoorCluster.append(int(data["ClusteringCoefficient"][key]))
        # xnew = np.linspace(min(XcoorCluster),max(XcoorCluster),300) #300 represents number of points to make between T.min and T.max

        # power_smooth = spline(XcoorCluster,YcoorCluster,xnew)
        plt.figure(6)
        plt.title('Plot showing the Distribution of the nodes with equal ClusteringCoefficient')
        #plt.plot(xnew,power_smooth, color=ColorScheme[datasetName])
        plt.plot(XcoorCluster, YcoorCluster, color=ColorScheme[datasetName])
        plt.ylabel('Occurences of the nodes with equal ClusteringCoefficient')
        plt.xlabel('ClusteringCoefficient of the node')



    for current in plt.get_fignums():


        plt.figure(current)
        # Remove the plot frame lines. They are unnecessary chartjunk.
        ax = plt.subplot(111)
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        if current == 1:
            pass
        elif current == 2:
            ax.set_yscale("log")
        else:
            ax.set_yscale("log")
            ax.set_xscale("log")

        if(current > 3):
            ax.legend(labels2)
        else:
            ax.legend(labels1)

        # Ensure that the axis ticks only show up on the bottom and left of the plot.
        # Ticks on the right and top of the plot are generally unnecessary chartjunk.
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        plt.savefig("plots/"+Sample+'figure%d.png' % current, bbox_inches="tight")


In [5]:



def tabling(FileListInconsistency, FileListKB):
    print("""\\begin{table}[ht]
\\begin{tabular}{|l|l|l|l|}
\\hline
           & Amount of 'Anti-patterns' & sum of 'Anti-patterns' &  Largest Inconsistency  \\\\ \\hline \\hline """)

    for file in FileListInconsistency:
        dataset = file.split("/")[-2]
        json_file = open(file, 'r')
        data = json.load(json_file)
        IDCoor = []
        SizeCoor = []
        CountCoor = []
        for elem in data:
            CountCoor.append(elem["Count"])
            SizeCoor.append(elem["Size"])
            IDCoor.append(elem["id"])

        AmountOfAntiPatterns = sum(i > 0 for i in CountCoor)

        print("     "+" & ".join([str(dataset),str(AmountOfAntiPatterns),str(sum(CountCoor)),str(max(SizeCoor))])+"\\\\ \\hline")

    print("""\\end{tabular}
\\end{table}""")

    print("")

    print("""\\begin{table}[ht]
\\begin{tabular}{|l|l|l|l|l|}
\\hline
           & Size & Expressivity & Namespaces & Distinct predicates \\\\ \\hline \\hline""")

    for file in FileListKB:
        dataset = file.split("/")[-2]
        json_file = open(file, 'r')
        data = json.load(json_file)
        print("     "+" & ".join([str(dataset),str(data["Size"]),str(data["Expressivity"]),str(len(data["Predicates"])),str(len(data["Namespaces"]))])+"\\\\ \\hline")
    print("""\\end{tabular}
\\end{table}""")


In [10]:
plots.plotting(FileListInconsistencySample, FileListKBSample, "Sample")

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [11]:
plots.plotting(FileListInconsistency, FileListKB, "")

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [8]:
tables.tabling(FileListInconsistency, FileListKB)

\begin{table}[ht]
\begin{tabular}{|l|l|l|l|}
\hline
           & Amount of 'Anti-patterns' & sum of 'Anti-patterns' &  Largest Inconsistency  \\ \hline \hline 
     DBpedia & 13 & 101349 & 19\\ \hline
     Yago & 135 & 1808977348 & 19\\ \hline
     wordnet & 0 & 0 & 19\\ \hline
     dblp-2012-11-28b & 0 & 0 & 19\\ \hline
     swdf & 0 & 0 & 19\\ \hline
     LOD & 222 & 1107375273 & 19\\ \hline
\end{tabular}
\end{table}

\begin{table}[ht]
\begin{tabular}{|l|l|l|l|l|}
\hline
           & Size & Expressivity & Namespaces & Distinct predicates \\ \hline \hline
     DBpedia & 1040358853 & EL+ & 20 & 18\\ \hline
     Yago & 158991568 & EL+ & 11 & 5\\ \hline
     wordnet & 5558748 & EL & 5 & 5\\ \hline
     dblp-2012-11-28b & 55586971 & EL & 4 & 7\\ \hline
     swdf & 242256 & EL & 60 & 22\\ \hline
\end{tabular}
\end{table}


In [9]:
tables.tabling(FileListInconsistencySample, FileListKBSample)

\begin{table}[ht]
\begin{tabular}{|l|l|l|l|}
\hline
           & Amount of 'Anti-patterns' & sum of 'Anti-patterns' &  Largest Inconsistency  \\ \hline \hline 
     dbpedia2016-04en & 2 & 7081 & 19\\ \hline
     yago2s & 0 & 0 & 19\\ \hline
     wordnet31 & 0 & 0 & 19\\ \hline
     dblp-2012-11-28b & 0 & 0 & 19\\ \hline
     swdf & 0 & 0 & 19\\ \hline
\end{tabular}
\end{table}

\begin{table}[ht]
\begin{tabular}{|l|l|l|l|l|}
\hline
           & Size & Expressivity & Namespaces & Distinct predicates \\ \hline \hline
     dbpedia2016-04en & 83464390 & EL & 14 & 13\\ \hline
     yago2s & 62562330 & EL & 7 & 4\\ \hline
     wordnet31 & 769026 & EL & 4 & 4\\ \hline
     dblp-2012-11-28b & 16812580 & EL & 3 & 7\\ \hline
     swdf & 53942 & EL & 52 & 21\\ \hline
\end{tabular}
\end{table}
